In [1]:
import ROOT

Welcome to JupyROOT 6.14/04


In [5]:
import csv

Maybe don't need to load CSV. We can maybe use the zboson as a test case and then move to Higgs boson....

In [6]:
mass_map = {}
with open('zboson.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    for row in reader:
        masses = []
        
'''convert this code to python...
while(invariantMasses.getline(line, 256)):
    vector<double> masses
    string s(line)
    string delimiter = ","
    string lepton = s.substr(0, s.find(delimiter))
    string invariantMass = s.substr(s.find(delimiter) + 1)
    try:
        masses = mass_map.at(lepton)
        masses.push_back(stod(invariantMass))
        mass_map[lepton] = masses
    except (const out_of_range & oor):
        masses.push_back(stod(invariantMass))
        mass_map.insert(make_pair(lepton, masses))
'''


'convert this code to python...\nwhile(invariantMasses.getline(line, 256)):\n    vector<double> masses\n    string s(line)\n    string delimiter = ","\n    string lepton = s.substr(0, s.find(delimiter))\n    string invariantMass = s.substr(s.find(delimiter) + 1)\n    try:\n        masses = mass_map.at(lepton)\n        masses.push_back(stod(invariantMass))\n        mass_map[lepton] = masses\n    except (const out_of_range & oor):\n        masses.push_back(stod(invariantMass))\n        mass_map.insert(make_pair(lepton, masses))\n'

In [7]:
'''// Parameters for the histogram.
  int numBins = 100;
  float lowerXRange = 0.0f;
  float upperXRange = 260.0f;

  // Create my histograms.
  TH1C * zBosonHist = new TH1C("zbosonHist", "Histogram of Invariant Masses of Dilepton Decay",
                                numBins, lowerXRange, upperXRange);
  TH1C * zBosonElectronHist = new TH1C("zBosonElectronHist", "Histogram of Invariant Masses of Dielectron Decays",
                                numBins, lowerXRange, upperXRange);
  TH1C * zBosonMuonHist = new TH1C("zBosonMuonHist", "Histogram of Invariant Masses of Dimuon Decays",
                                numBins, lowerXRange, upperXRange);

  // Fill the histograms
  map<string, vector<double>>::iterator it;
  for(it = mass_map.begin(); it != mass_map.end(); it++) {
    vector<double> masses = it->second;
    for(unsigned int i = 0; i < masses.size(); ++i) {
      zBosonHist->Fill(masses[i]);
      if(it->first.compare("e")) {
        zBosonElectronHist->Fill(masses[i]);
      } else if(it->first.compare("m")) {
        zBosonMuonHist->Fill(masses[i]);
      }
    }
  }

  // Analyze the histograms.
  printf("=================================================\n");
  printf("Analyzing all of the dilepton decays...\n");
  analyzeZBosonHistogram("overallHistCanvas", zBosonHist);
  printf("=================================================\n");
  printf("Analyzing just the dielectron decays\n");
  analyzeZBosonHistogram("dielectronHistCanvas", zBosonElectronHist);
  printf("=================================================\n");
  printf("Analyzing just the dimuon decays\n");
  analyzeZBosonHistogram("dimuonHistCanvas", zBosonMuonHist);
  printf("=================================================\n");'''

'// Parameters for the histogram.\n  int numBins = 100;\n  float lowerXRange = 0.0f;\n  float upperXRange = 260.0f;\n\n  // Create my histograms.\n  TH1C * zBosonHist = new TH1C("zbosonHist", "Histogram of Invariant Masses of Dilepton Decay",\n                                numBins, lowerXRange, upperXRange);\n  TH1C * zBosonElectronHist = new TH1C("zBosonElectronHist", "Histogram of Invariant Masses of Dielectron Decays",\n                                numBins, lowerXRange, upperXRange);\n  TH1C * zBosonMuonHist = new TH1C("zBosonMuonHist", "Histogram of Invariant Masses of Dimuon Decays",\n                                numBins, lowerXRange, upperXRange);\n\n  // Fill the histograms\n  map<string, vector<double>>::iterator it;\n  for(it = mass_map.begin(); it != mass_map.end(); it++) {\n    vector<double> masses = it->second;\n    for(unsigned int i = 0; i < masses.size(); ++i) {\n      zBosonHist->Fill(masses[i]);\n      if(it->first.compare("e")) {\n        zBosonElectronHist->

In [8]:
'''  
TF1 * fallingExponentialFit = new TF1("fallingExponentialFit", "[0]*exp(-1*[1]*x)", 0, 75);
fallingExponentialFit->SetParameters(100, 0.01);
fallingExponentialFit->SetParLimits(0, 0.01, 10000);
fallingExponentialFit->SetParLimits(1, 0.00001, 100);

TF1 * gaussianFit = new TF1("gaussianFit", "[0]*exp(-0.5*((x-[1])/[2])^2)", 75, 110);
gaussianFit->SetParameters(100, 100, 20);
gaussianFit->SetParLimits(0, 0, 1000);
gaussianFit->SetParLimits(1, 50, 150);
gaussianFit->SetParLimits(2, 0.1, 100);
gaussianFit->SetLineColor(kGreen);

TCanvas * canvas = new TCanvas(canvasName.c_str(), canvasName.c_str(), 700, 700);
histogram->Fit(fallingExponentialFit, "LR+");
histogram->Fit(gaussianFit, "LR+");

ROOT::Math::WrappedTF1 wrappedFallExp(*fallingExponentialFit);
ROOT::Math::WrappedTF1 wrappedGauss(*gaussianFit);
ROOT::Math::GaussIntegrator integrator;
integrator.SetFunction(wrappedFallExp);
integrator.SetRelTolerance(0.001);
double noiseIntegration = integrator.Integral(75, 125);
integrator.SetFunction(wrappedGauss);
double signalIntegration = integrator.Integral(75, 125);
double _signal = signalIntegration - noiseIntegration;
double snr = _signal/noiseIntegration;

printf("The measured noise within the signal region was: %f\n", noiseIntegration);
printf("The measured signal within the signal region was: %f\n", _signal);
printf("The signal to noise ratio (SNR) is: %f\n", snr);
printf("=====\n");
printf("The mass of the Z boson is %f +- %f GeV/c^2\n", gaussianFit->GetParameter(1), gaussianFit->GetParError(1));
histogram->Draw();
string pngFilename = canvasName.append(".png");
canvas->SaveAs(pngFilename.c_str());'''

'  \nTF1 * fallingExponentialFit = new TF1("fallingExponentialFit", "[0]*exp(-1*[1]*x)", 0, 75);\nfallingExponentialFit->SetParameters(100, 0.01);\nfallingExponentialFit->SetParLimits(0, 0.01, 10000);\nfallingExponentialFit->SetParLimits(1, 0.00001, 100);\n\nTF1 * gaussianFit = new TF1("gaussianFit", "[0]*exp(-0.5*((x-[1])/[2])^2)", 75, 110);\ngaussianFit->SetParameters(100, 100, 20);\ngaussianFit->SetParLimits(0, 0, 1000);\ngaussianFit->SetParLimits(1, 50, 150);\ngaussianFit->SetParLimits(2, 0.1, 100);\ngaussianFit->SetLineColor(kGreen);\n\nTCanvas * canvas = new TCanvas(canvasName.c_str(), canvasName.c_str(), 700, 700);\nhistogram->Fit(fallingExponentialFit, "LR+");\nhistogram->Fit(gaussianFit, "LR+");\n\nROOT::Math::WrappedTF1 wrappedFallExp(*fallingExponentialFit);\nROOT::Math::WrappedTF1 wrappedGauss(*gaussianFit);\nROOT::Math::GaussIntegrator integrator;\nintegrator.SetFunction(wrappedFallExp);\nintegrator.SetRelTolerance(0.001);\ndouble noiseIntegration = integrator.Integral(75,